# Lecture 20 – Data 100, Summer 2024

Data 100, Summer 2024

[Acknowledgments Page](https://ds100.org/su24/acks/)

## Starting Up SQL

Before we look at SQL syntax in detail, let's first get ourselves set up to run SQL queries in Jupyter.

### Approach #1: SQL Magic

**1. Load the `sql` Module.** 

Load `%%sql` cell magic.

In [ ]:
%load_ext sql

**2. Connect to a database.**  

Here, we connect to the SQLite database `basic_examples.db`.

In [ ]:
%sql duckdb:///data/example_duck.db --alias duck

If you were connecting to an "enterprise data platform"

```python
from sqlalchemy import create_engine

snow_engine = create_engine(
    f"snowflake://{user}:{password}@{account_identifier}")
%sql snow_engine --alias snow

db_engine = create_engine(
  url = f"databricks://token:{access_token}@{server_hostname}?" +
        f"http_path={http_path}&catalog={catalog}&schema={schema}"
)
%sql db_engine --alias db
```

<br/>

**3. Run a simple SQL query.** 

Note the `%%sql` lets Jupyter parse the rest of the lines as a SQL command.

In [ ]:
%%sql
SELECT * FROM Dragon;

**Simple query, this time on two different lines.**

In [ ]:
%%sql
SELECT *
FROM Dragon;

### Approach #2: `pd.read_sql`

It turns out that `pandas` has a special-purpose function to parse SQL queries. We can pass in a SQL query as a string to return a `pandas` DataFrame. To achieve the same result as we did using cell magic above, we can do the following.

**1. Connect to a database**

In [ ]:
import sqlalchemy 
import pandas as pd

engine = sqlalchemy.create_engine("duckdb:///data/example_duck.db")


**2. Run a simple SQL query**

In [ ]:
query = """
SELECT * 
FROM Dragon;
"""

df = pd.read_sql(query, engine)
df

### Approach "3" -- Duck DB Special

Now that we are using DuckDB we can do something extra crazy:

In [ ]:
import seaborn as sns
mpg = sns.load_dataset("mpg")

In [ ]:
%%sql
SELECT * FROM mpg

That is right!! DuckDB can also see my dataframes in the python environment allowing me to do dataframe manipulation in SQL!

<br><br>
**Return to Lecture**

---

## Working with the `sql` results as Python variables

By default, executing a query with a magic command produces output but doesn't save it into any Python variable:

In [ ]:
%%sql 
SELECT * FROM Dragon

This can be inconvenient if you later want to do further processing of these data in Python.

### Storing one-line `%sql` queries

For simple one-line queries, you can use IPython's ability to store the result of a magic command like `%sql` as if it were any other Python statement, and save the output to a variable:

In [ ]:
dragon_table = %sql SELECT * FROM Dragon
dragon_table

As noted above, the result of the query is a Python variable of type `ResultSet`, more specifically:

In [ ]:
type(dragon_table)

You need to manually convert it to a Pandas DataFrame if you want to do pandas-things with its content:

In [ ]:
dragon_df = dragon_table.DataFrame()
dragon_df

You can configure `jupysql` to _automatically_ convert all outputs to Pandas DataFrames. This can be handy if you intend all your Python-side work to be done with Pandas, as it saves you from manually having to call `.DataFrame()` first on all outputs. On the other hand, you don't get access to the original SQL `ResultSet` object, which have a number of interesting properties and capabilities. You can learn more about those in the [jupysql documentation](https://jupysql.ploomber.io).

For now, let's turn this on so you can see how this simplified, "pandas all the way" worfklow looks like:

In [ ]:
%config SqlMagic.autopandas = True

In [ ]:
dragon_df = %sql SELECT * FROM Dragon
dragon_df

In [ ]:
type(dragon_df)

### Storing multi-line `%%sql` queries

For a more complex query that won't fit in one line, such as for example:

In [ ]:
%%sql
SELECT year, COUNT(*)
FROM Dragon
GROUP BY year;

You can use the `variable <<` syntax in jupysql to store its output (this will honor your `autopandas` state and store either a `sql.run.ResultState` or a Pandas `DataFrame`):

In [ ]:
%%sql dragon_years <<
SELECT year, COUNT(*)
FROM Dragon
GROUP BY year;

In [ ]:
dragon_years


---

## Tables and Schema

A **database** contains a collection of SQL **tables**. Let's connect to our "toy" database `example_duck.db` and explore the tables it stores.

In [ ]:
%%sql
SELECT * FROM information_schema.tables

In [ ]:
%%sql
SELECT * FROM information_schema.columns

### Getting Schema information with SQLAlchemy 
How you list the tables varies across database platforms.  For example, the statement:

```sql
SELECT * FROM information_schema.columns
```

only works on Postgres compatible databases.

For example, if we wanted to get the schema for tables in sqlite we would need the following:

In [ ]:
pd.read_sql("SELECT * FROM sqlite_schema", "sqlite:///data/basic_examples.db")

Fortunately, SQLAlchemy has some generic tools that will be helpful regardless of what database platform you use.

In [ ]:
from sqlalchemy import inspect
inspector = inspect(engine)
inspector.get_table_names()

In [ ]:
inspector.get_columns('scene')

Same with SQLite

In [ ]:
sqlite_engine = sqlalchemy.create_engine("sqlite:///data/basic_examples.db")
inspect(sqlite_engine).get_columns("scene")


Example of table creation with interesting constraints

More advanced example of creating tables with primary and foreign key constraints:

In [ ]:
%%sql

DROP TABLE IF EXISTS grade;
DROP TABLE IF EXISTS assignment;
DROP TABLE IF EXISTS student;


CREATE TABLE student (
    student_id INTEGER PRIMARY KEY,
    name VARCHAR,
    email VARCHAR
);

CREATE TABLE assignment (
    assignment_id INTEGER PRIMARY KEY,
    description VARCHAR
);

CREATE TABLE grade (
    student_id INTEGER,
    assignment_id INTEGER,
    score REAL CHECK (score > 0 AND score <= 100),
    FOREIGN KEY (student_id) REFERENCES student(student_id),
    FOREIGN KEY (assignment_id) REFERENCES assignment(assignment_id)
);

INSERT INTO student VALUES
(123, 'JoeyG', 'jegonzal@berkeley.edu'),
(456, 'NargesN', 'norouzi@berkeley.edu');

INSERT INTO assignment VALUES
(1, 'easy assignment'),
(2, 'hard assignment');



In [ ]:
%%sql 
INSERT INTO grade VALUES
(123, 1, 80),
(123, 2, 42),
(456, 2, 100);



In [ ]:
%sql SELECT * FROM grade;

<br/><br/><br/>

---

## Basic Queries

Every SQL query *must* contain a `SELECT` and `FROM` clause.

* `SELECT`: specify the column(s) to return in the output
* `FROM`: specify the database table from which to extract data

In [ ]:
%%sql
SELECT * FROM Dragon;

In [ ]:
%%sql
SELECT cute, year FROM Dragon;

**Aliasing** with `AS`

In [ ]:
%%sql
SELECT cute AS cuteness,
       year AS "birth year"
FROM Dragon;

**Uniqueness** with `DISTINCT`

In [ ]:
%%sql
SELECT DISTINCT year
FROM Dragon;

**Filtering** with `WHERE`

In [ ]:
%%sql
SELECT name, year
FROM Dragon
WHERE cute > 0;

In [ ]:
%%sql
SELECT name, cute, year
FROM Dragon
WHERE cute > 0 OR year > 2013;

In [ ]:
%%sql
SELECT name, year
FROM Dragon 
WHERE name IN ('puff', 'hiccup');

In [ ]:
%%sql
SELECT name, cute
FROM Dragon
WHERE cute IS NOT NULL;

**Ordering** data using `ORDER BY`

In [ ]:
%%sql
SELECT *
FROM Dragon
ORDER BY cute DESC;

**Restricting** output with `LIMIT` and `OFFSET`

In [ ]:
%%sql
SELECT *
FROM Dragon
LIMIT 2;

In [ ]:
%%sql
SELECT *
FROM Dragon
LIMIT 2
OFFSET 1;

What if we wanted a random sample:

In [ ]:
%%sql
SELECT *
FROM Dragon
ORDER BY RANDOM() 
LIMIT 2

In [ ]:
%%sql
SELECT * 
FROM Dragon USING SAMPLE reservoir(2 ROWS) REPEATABLE (100);

## Grouping Data with `GROUP BY`

In [ ]:
%%sql
SELECT *
FROM Dish;

A small note: the fact that `type` is highlighted in green below is a consequence of Jupyter assuming that we are writing Python code (where `type` is a built-in keyword). `type` does *not* have a special meaning in SQL, so the color below does not indicate any special functionality. When we run the cell, Jupyter realizes it should recognize the code as SQL. 

In [ ]:
%%sql
SELECT type
FROM Dish;

In [ ]:
%%sql
SELECT type
FROM Dish
GROUP BY type;

In [ ]:
%%sql
SELECT type, SUM(cost)
FROM Dish
GROUP BY type;

In [ ]:
%%sql
SELECT type, 
       SUM(cost), 
       MIN(cost),
       MAX(name)
FROM Dish
GROUP BY type;

In [ ]:
%%sql
SELECT year, COUNT(cute)
FROM Dragon
GROUP BY year;

In [ ]:
%%sql
SELECT year, COUNT(*)
FROM Dragon
GROUP BY year;

### Loading More Data

Add support for reading sqlite tables using duckdb

In [ ]:
import duckdb
conn = duckdb.connect()
conn.query("INSTALL sqlite")
%config SqlMagic.displaylimit = 100

Loading the SQLite database file using duckdb.

In [ ]:
%sql duckdb:///data/basic_examples.db --alias basic

In [ ]:
pd.read_sql("SELECT * FROM sqlite_schema", "duckdb:///data/basic_examples.db")

Get the large IMDB database file 

In [ ]:
import os
if os.path.exists("/home/jovyan/shared/sql/imdb_duck.db"):
    imdbpath = "duckdb:////home/jovyan/shared/sql/imdb_duck.db"
elif os.path.exists("data/imdb_duck.db"):
    imdbpath =  "duckdb:///data/imdb_duck.db"
else:
    import gdown
    url = 'https://drive.google.com/uc?id=10tKOHGLt9QoOgq5Ii-FhxpB9lDSQgl1O'
    output_path = 'data/imdb_duck.db'
    gdown.download(url, output_path, quiet=False)
    imdbpath = "duckdb:///data/imdb_duck.db"
print(imdbpath)

In [ ]:
from sqlalchemy import create_engine
imdb_engine = create_engine(imdbpath, connect_args={'read_only': True})
%sql imdb_engine --alias imdb

### Filtering Groups Using `HAVING`

In [ ]:
%%sql basic
    
SELECT *
FROM Dish;

In [ ]:
%%sql
    
SELECT type, COUNT(*)
FROM Dish
GROUP BY type
HAVING MAX(cost) < 8;

In [ ]:
%%sql basic
    
SELECT type, COUNT(*)
FROM Dish
WHERE cost < 8
GROUP BY type

In [ ]:
%%sql basic
SELECT type, MAX(name)
FROM DishDietary
WHERE notes == 'gf'
GROUP BY type
HAVING MAX(cost) <= 7;


### EDA in SQL

Our typical workflow when working with "big data" is:
* Use SQL to query data from a database
* Use Python (with `pandas`) to analyze this data in detail

We can, however, still perform simple data cleaning and re-structuring using SQL directly. To do so, we'll consider the `Title` table from the IMDB dataset.

In [ ]:
%%sql imdb
SELECT setseed(0.42); -- Setting the random number seed


SELECT *
FROM Title
ORDER BY RANDOM()
LIMIT 10;

#### Matching Text Using `LIKE`

In [ ]:
%%sql imdb 
    
SELECT titleType, primaryTitle
FROM Title
WHERE primaryTitle LIKE '%Star Wars%'

In [ ]:
%%sql imdb 
    
SELECT titleType, primaryTitle
FROM Title
WHERE primaryTitle SIMILAR TO '.*Star Wars.*'

In [ ]:
%%sql imdb
    
SELECT titleType, primaryTitle
FROM Title
WHERE primaryTitle LIKE 'Harry Potter and the Deathly Hallows: Part _'

#### Converting Data Types Using `CAST`

In [ ]:
%%sql imdb
    
SELECT primaryTitle, CAST(runtimeMinutes AS REAL)
FROM Title
LIMIT 10;

### Applying Conditions With `CASE`

Here, we return a random order so we can see the various movie ages (otherwise, the top few entries happen to all be old movies).

In [ ]:
%%sql imdb

SELECT setseed(0.42); -- Setting the random number seed
    
SELECT titleType, startYear,
CASE WHEN startYear < '1950' THEN 'old'
     WHEN startYear < '2000' THEN 'mid-aged'
     ELSE 'new'
     END AS movie_age
FROM Title
ORDER BY RANDOM()
LIMIT 10;

<br><br><br>

---


## Joining Tables

We combine data from multiple tables by performing a **join**. We will explore joins using the cats database, which includes two tables: `s` and `t`.

In [ ]:
%%sql basic
SELECT * FROM s;

In [ ]:
%%sql basic
SELECT * FROM t;

#### Inner Join

In [ ]:
%%sql basic
SELECT s.id, name, breed
FROM s INNER JOIN t ON s.id = t.id;

By default, `JOIN`ing without specifying a join type will default to an inner join.

In [ ]:
%%sql basic
SELECT s.id, name, breed
FROM s JOIN t ON s.id = t.id;

### Cross Join

In [ ]:
%%sql basic
SELECT *
FROM s CROSS JOIN t;

Equivalently, a cross join can be performed using the following syntax:

In [ ]:
%%sql basic
SELECT *
FROM s, t

Conceptually, an inner join is equivalent to a cross join where irrelevant rows are removed.

In [ ]:
%%sql basic
    
SELECT *
FROM s CROSS JOIN t
WHERE s.id = t.id;

#### Left Outer Join

In [ ]:
%%sql basic
    
SELECT *
FROM s LEFT JOIN t ON s.id = t.id;

#### Right Outer Join

In [ ]:
%%sql basic
    
SELECT *
FROM s RIGHT JOIN t ON s.id = t.id;

#### Full Outer Join

In [ ]:
%%sql basic
    
SELECT *
FROM s FULL  JOIN t ON s.id = t.id;

#### Aliasing in Joins

Let's return to the IMDB dataset. Now, we'll consider two tables: `Title` and `Rating`.

When working with tables that have long names, we often create an **alias** using the `AS` keyword (much like we did with columns in the previous lecture). This makes it easier to reference these tables when performing a join.

In [ ]:
%%sql imdb
    
SELECT primaryTitle, averageRating, genres, 
FROM Title AS T INNER JOIN Rating AS R
    ON T.tconst = R.tconst
WHERE numVotes > 100 
ORDER BY averageRating DESC
LIMIT 10;

The `AS` keyword is optional – omitting it will still create an alias. We typically include `AS` to make our queries more readable.

In [ ]:
%%sql imdb 
    
SELECT primaryTitle, averageRating
FROM Title T INNER JOIN Rating R
    ON T.tconst = R.tconst
WHERE numVotes > 100 
ORDER BY averageRating DESC
LIMIT 10;

Referencing columns using the full or aliased table name is important to avoid ambiguity. Suppose the tables we are trying to join both include a column with the same name, like the `tconst` columns present in both the `Title` and `Rating` tables of the IMDB database. If we do not specify which table's column we wish to reference, SQL will not be able to process our query.

In the cell below, it is unclear if we are referring to the `tconst` column from the `Title` table or the `tconst` column from the `Rating` table. SQL errors.

```sql
SELECT primaryTitle, averageRating
FROM Title AS T INNER JOIN Rating AS R
ON tconst = tconst;
```

## IMDB Case Study

Let's perform our first data science task in SQL – we'll explore the IMDB dataset in further detail.

First, let's understand what tables are present in the database.

In [ ]:
%%sql imdb

SELECT tbl_name, sql FROM sqlite_master WHERE type='table'

Let's take a closer look at the `Title` table. Below, we preview a few rows of interest.

In [ ]:
%%sql imdb 
    
SELECT *
FROM Title
WHERE primaryTitle IN ('Ginny & Georgia', 'What If...?', 'Succession', 'Veep', 'Tenet')
LIMIT 50;

There are a few issues with how this data is stored. Can you spot them?

* We see some rows with **missing** `runtimeMinutes`. This suggests malformed data. We could drop these rows in `pandas` using `[]` or `.dropna()`.

Instead, let's see how we can can do these fixes in SQL. 

In [ ]:
%%sql imdb 
    
SELECT 
    tconst AS id,
    primaryTitle AS title, 
    titleType,
    runtimeMinutes,
    startYear
FROM Title
WHERE 
    runtimeMinutes > 0 AND runtimeMinutes < 300 -- get rid of outliers and missing values --
    AND startYear > 0     -- get rid of missing values --
LIMIT 10

Perhaps we're interested in only **top rated action movies**. We can refine our query results futher.

In [ ]:
%%sql imdb
    
SELECT 
    T.tconst AS id,
    primaryTitle AS title, 
    titleType,
    runtimeMinutes,
    startYear,
    genres,
    averageRating, 
    numVotes
FROM Title T, Rating R
WHERE 
    T.tconst = R.tconst 
    AND runtimeMinutes > 0 AND runtimeMinutes < 300 -- get rid of outliers and missing values --
    AND startYear > 0     -- get rid of missing values --
    AND titleType = 'movie' -- get movies
    AND numVotes >= 1000
    AND genres LIKE '%Action%'
ORDER BY averageRating DESC
LIMIT 10

### Using CTEs to do some complex analysis  
Here I will use a common table expression (CTE) to construct a table that I use as an input in another query.

This is done with the syntax:

```sql
WITH table_name as ( 
    SELECT ...
)
SELECT ... 
FROM table_name, ...
...    
```

In [ ]:
%%sql
WITH good_action_movies AS (
    SELECT *
    FROM Title T JOIN Rating R ON T.tconst = R.tconst  
    WHERE genres LIKE '%Action%' AND averageRating > 7 AND numVotes > 5000
),
prolific_actors AS (
    SELECT N.nconst, primaryName, COUNT(*) as numRoles
    FROM Name N JOIN Principal P ON N.nconst = P.nconst
    WHERE category = 'actor'
    GROUP BY N.nconst, primaryName
)
SELECT primaryTitle, primaryName, numRoles, ROUND(averageRating) AS rating
FROM good_action_movies m, prolific_actors a, principal p
WHERE p.tconst = m.tconst AND p.nconst = a.nconst
ORDER BY rating DESC, numRoles DESC
LIMIT 10



### How do ratings vary through the years?

If we want to know the critic ratings for each of these action movies, we can join `Title` with the `Rating` table.

In the cell below, we use the **`<<` operator** to store the result of our query as a variable.

In [ ]:
%sql imdb
%config SqlMagic.autopandas = True

In [ ]:
%%sql action_movies_df << 
SELECT 
    T.tconst AS id,
    primaryTitle AS title, 
    titleType,
    runtimeMinutes AS time,
    startYear AS year,
    genres,
    averageRating AS rating, 
    numVotes
FROM Title T, Rating R
WHERE 
    T.tconst = R.tconst 
    AND time > 0 AND time < 300 -- get rid of outliers and missing values --
    AND year > 0     -- get rid of missing values --
    AND titleType = 'movie' -- get movies
    AND numVotes >= 1000
    AND genres LIKE '%Action%'
ORDER BY startYear

In [ ]:
action_movies_df

In [ ]:
import plotly.express as px
px.scatter(action_movies_df, x = "year", y="rating", opacity=0.5)

Using a common table expression (CTE):

In [ ]:
%%sql action_grouped_df <<

WITH action_movies AS (
    SELECT 
        T.tconst AS id,
        primaryTitle AS title, 
        titleType,
        runtimeMinutes AS time,
        startYear AS year,
        genres,
        averageRating AS rating, 
        numVotes
    FROM Title T, Rating R
    WHERE 
        T.tconst = R.tconst 
        AND time > 0 AND time < 300 -- get rid of outliers and missing values --
        AND year > 0     -- get rid of missing values --
        AND titleType = 'movie' -- get movies
        AND numVotes >= 1000
    AND genres LIKE '%Action%'
)
SELECT 
    year, 
    median(rating) AS median_rating,
    quantile_cont(rating, .25) AS lower_q,
    quantile_cont(rating, .75) AS upper_q,
FROM action_movies
GROUP BY year
ORDER BY year

Let's look at what we got.

In [ ]:
import plotly.express as px
action_grouped_df['error_p'] = action_grouped_df['upper_q'] - action_grouped_df['median_rating']
action_grouped_df['error_m'] = action_grouped_df['median_rating'] - action_grouped_df['lower_q'] 
fig = px.line(action_grouped_df, x="year", y="median_rating", 
              error_y="error_p", error_y_minus="error_m")
fig

### How does the runtime of an action movie influence the rating it receives?

In [ ]:
px.scatter(action_movies_df, x="time", y="rating", opacity=0.1)

In [ ]:
px.density_heatmap(action_movies_df, x="time", y="rating",nbinsx=100, nbinsy=100)

In [ ]:
%%sql action_grouped_df << 
WITH action_movies AS (
    SELECT 
        T.tconst AS id,
        primaryTitle AS title, 
        titleType,
        runtimeMinutes AS time,
        startYear AS year,
        genres,
        averageRating AS rating, 
        numVotes
    FROM Title T, Rating R
    WHERE 
        T.tconst = R.tconst 
        AND time > 0 AND time < 300 -- get rid of outliers and missing values --
        AND year > 0     -- get rid of missing values --
        AND titleType = 'movie' -- get movies
        AND numVotes >= 1000
    AND genres LIKE '%Action%'
)
SELECT 
    ceil(time / 10) * 10 as time_desc, 
    median(rating) AS median_rating,
    quantile_cont(rating, .25) AS lower_q,
    quantile_cont(rating, .75) AS upper_q,
FROM action_movies
GROUP BY time_desc
ORDER BY time_desc

In [ ]:
import plotly.express as px
action_grouped_df['error_p'] = action_grouped_df['upper_q'] - action_grouped_df['median_rating']
action_grouped_df['error_m'] = action_grouped_df['median_rating'] - action_grouped_df['lower_q'] 
fig = px.line(action_grouped_df, x="time_desc", y="median_rating", 
              error_y="error_p", error_y_minus="error_m")
fig

Could we use a movie's runtime to predict its rating?

In [ ]:
import sklearn.linear_model as lm
import numpy as np
X, Y = action_movies_df["time"].to_numpy(), action_movies_df["rating"].to_numpy()

model = lm.LinearRegression()
model.fit(X[..., np.newaxis], Y)

xs = np.linspace(X.min(),X.max())
yhats = model.predict(xs[:, np.newaxis])

In [ ]:
fig = px.scatter(x=X, y=Y, opacity=0.1)
fig.add_scatter(x=xs, y=yhats)

In [ ]:
px.scatter(x=X, y=Y, opacity=0.1, trendline="ols")

## More Advanced SQL with DuckDB

You could do a lot what we do in this class just using Duck DB and the many duck db functions:

https://duckdb.org/docs/sql/functions/overview

In [ ]:
from ds100_utils import fetch_and_cache

In [ ]:
url = "https://gist.github.com/domoritz/fd517a3a3a210c24a488e61870e2cf2c/raw/b1d53719e8e0eb9f6a95de82fdaccf0b001c0dea/flights-1m.parquet"
fetch_and_cache(url,"flights.parquet")

In [ ]:
%%sql
SELECT * FROM 'data/flights.parquet' LIMIT 10;

In [ ]:
%%sql avg_delays <<

SELECT 
    dayname(fl_date) AS "Day of the Week", 
    mean(dep_delay) AS "Mean Departure Delay", 
FROM 'data/flights.parquet'
GROUP BY "Day of the Week"

In [ ]:
import plotly.express as px
px.bar(avg_delays, x="Day of the Week", y = "Mean Departure Delay",
       category_orders={"Day of the Week": ["Sunday", "Monday", "Tuesday", 
                      "Wednesday", "Thursday", "Friday", "Saturday"]})